In [12]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.utils import get_column_letter # openpyxlで列番号をアルファベットに変換するために必要
import re #正規表現モジュール使用　日本語と半角を調べる用


# Excelファイルの読み込み（Sheet1を対象）
df_2022 = pd.read_excel('2022_年間売上表.xlsx', sheet_name='Sheet1')

df_2023 = pd.read_excel('2023_年間売上表.xlsx', sheet_name='Sheet1')


# データを連結
df_all = pd.concat([df_2022, df_2023], ignore_index=True)

# 「商品」「売上年」ごとに金額を合計
grouped_df = df_all.groupby(['商品', '売上年'], as_index=False)['金額（千円）'].sum()

# 新しいExcelファイルに書き込み
output_file = '売上集計表.xlsx'
grouped_df.to_excel(output_file, index=False)

# ヘッダーの色付け（#F2F2F2）処理
wb = load_workbook(output_file)
ws = wb.active

# ヘッダー行を薄いグレーに塗る
fill = PatternFill(start_color='F2F2F2', end_color='F2F2F2', fill_type='solid')
for cell in ws[1]:
    cell.fill = fill

# 列情報取得
num_cols = len(grouped_df.columns)

# 文字幅の見積り関数（日本語考慮）
def get_adjusted_length(value):
    text = str(value)
    # 全角文字（漢字・ひらがな・カタカナなど）は幅2、半角は幅1と仮定
    return sum(2 if re.match(r'[^\x00-\x7F]', ch) else 1 for ch in text)

# 各列に対して最大の文字幅を計算して列幅設定
for col_num in range(1, num_cols + 1):
    max_length = 0
    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=col_num, max_col=col_num):
        for cell in row:
            if cell.value:
                max_length = max(max_length, get_adjusted_length(cell.value))
    adjusted_width = (max_length + 2)
    ws.column_dimensions[get_column_letter(col_num)].width = adjusted_width

# 「金額」列の列番号を取得（ヘッダーから検索）
for col in range(1, ws.max_column + 1):
    if ws.cell(row=1, column=col).value == '金額（千円）':
        amount_col_letter = get_column_letter(col)
        for row in range(2, ws.max_row + 1):  # データ行に適用
            ws[f'{amount_col_letter}{row}'].number_format = '#,##0'
        break

# 保存
wb.save(output_file)
print(f"処理が完了しました: {output_file}")


処理が完了しました: 売上集計表.xlsx
